In [3]:
import pandas as pd
import json

In [4]:
with open('mysql.json', 'r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [5]:
import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

### 패스워드 암호화

In [6]:
import hashlib

pwd ='1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

### base64로 인코딩

In [7]:
import base64

base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [11]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

### 패스워드를 생성하는 함수

In [12]:
import hashlib, base64

def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd

In [13]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

### users table에 패스워드 넣기
- uid + 21

In [14]:
sql = 'select uid from users'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('mji',),
 ('rjh',),
 ('twonkim',),
 ('wjlee',))

In [15]:
sql_update = 'update users set pwd=%s where uid=%s;'

for result in results:
    uid = result[0]
    pwd = gen_pwd(uid+'12')
    cur.execute(sql_update,(pwd, uid))

conn.commit()


- email : uid@mc.com

In [16]:
sql_email_update = 'update users set email=%s where uid=%s;'

for result in results:
    uid = result[0]
    email = uid+'@mc.com'
    cur.execute(sql_email_update,(email,uid))

conn.commit()

### log in

#### uid가 없는 경우

In [17]:
uid = 'fool'
sql_login = 'select uid, pwd from users where uid=%s and is_deleted=0'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
result

#### 비밀번호가 틀린 경우

In [18]:
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
result

('djy', '3ei6D4xFNgWfKckW1H0EnacUZBWtH1UnqXKGoekcq1Q=')

In [19]:
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('비밀번호가 틀렸습니다.')

비밀번호가 틀렸습니다.


#### OK

In [30]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('비밀번호가 틀렸습니다.')

비밀번호가 틀렸습니다.


In [31]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd==hashed_pwd:
        print('Login')
    else:
        print('비밀번호가 틀렸습니다.')
else:
    print('Bad uid')

비밀번호가 틀렸습니다.


#### 예외 처리를 통해 해결하는 경우

In [32]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
try:
    if result:
        db_pwd = result[1]
        hashed_pwd = gen_pwd(pwd)
        if db_pwd==hashed_pwd:
            print('Login')
        else:
            print('비밀번호가 틀렸습니다.')
except:
    print('Bad uid')

비밀번호가 틀렸습니다.


### 마무리

In [33]:
cur.close()
conn.close()